In [5]:
import geopandas as gpd
import pandas as pd


In [6]:
shake = gpd.read_file("Data/shakemap.geojson")

shake_utm = shake.to_crs(epsg=32645)

# 3) Buffer safely in meters (3 km)
shake_utm["geometry"] = shake_utm.geometry.buffer(3000)

# 4) Back to lat/lon for joining later
shake_polys = shake_utm.to_crs(epsg=4326)
shake

,value,units,color,weight,geometry
0,3.5,intensity,#90f3ff,2,"MULTILINESTRING ((87.6154 25.1848, 87.6272 25...."
1,4.0,intensity,#80ffff,1,"MULTILINESTRING ((84.67071 25.1848, 84.67631 2..."
2,4.5,intensity,#7dffc9,2,"MULTILINESTRING ((86.3772 28.96854, 86.39387 2..."
3,5.0,intensity,#7aff93,1,"MULTILINESTRING ((86.39387 28.82304, 86.39561 ..."
4,5.5,intensity,#bdff4a,2,"MULTILINESTRING ((85.8772 28.15704, 85.89387 2..."
5,6.0,intensity,#ffff00,1,"MULTILINESTRING ((85.8772 28.03972, 85.89387 2..."
6,6.5,intensity,#ffe300,2,"MULTILINESTRING ((85.99387 27.95432, 86.01053 ..."
7,7.0,intensity,#ffc800,1,"MULTILINESTRING ((85.9772 27.87287, 85.99387 2..."
8,7.5,intensity,#ffad00,2,"MULTILINESTRING ((85.9272 27.7253, 85.94387 27..."


In [7]:
wards = gpd.read_file("Data/nepal-wards.geojson")

In [8]:
wards = wards.to_crs(epsg=32645)
shake_polys = shake_polys.to_crs(epsg=32645)


In [9]:
wards["centroid"] = wards.geometry.centroid
wards = wards.set_geometry("centroid")

ward_intensity = gpd.sjoin_nearest(
    wards,
    shake_polys[["value", "geometry"]],
    how="left",
)
ward_intensity = ward_intensity.rename(columns={"value": "intensity"})

In [10]:
district_intensity = (
    ward_intensity.groupby("DIST_ID")["intensity"]
    .mean()
    .reset_index()
)

district_intensity = district_intensity.rename(columns={"DIST_ID": "district_id"})
district_intensity


,district_id,intensity
0,1,4.000000
1,2,4.000000
2,3,4.000000
3,4,4.280000
4,5,4.643939
...,...,...
70,71,3.500000
71,72,3.500000
72,73,3.500000
73,74,3.500000


In [11]:
import pandas as pd

build = pd.read_csv("Data/Nepal_buildings_w.csv")

district_intensity["district_id"] = district_intensity["district_id"].astype(int)
build["district_id"] = build["district_id"].astype(int)



build = build.merge(
    district_intensity,
    on="district_id",
    how="left"
)

In [12]:
print(district_intensity[["district_id", "intensity"]])


    district_id  intensity
0             1   4.000000
1             2   4.000000
2             3   4.000000
3             4   4.280000
4             5   4.643939
..          ...        ...
70           71   3.500000
71           72   3.500000
72           73   3.500000
73           74   3.500000
74           75   3.500000

[75 rows x 2 columns]


In [13]:
print(build[["district_id", "intensity"]])

        district_id  intensity
0                12   5.339286
1                12   5.339286
2                12   5.339286
3                12   5.339286
4                12   5.339286
...             ...        ...
762101           36   4.111940
762102           36   4.111940
762103           36   4.111940
762104           36   4.111940
762105           36   4.111940

[762106 rows x 2 columns]


In [14]:
build.to_csv("Data/Nepal_building_with_Intensity.csv", index=False)